#### merging and combining all data

In [5]:
import os
import pandas as pd
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Emory/NFL Lab/data/')

In [6]:
all_weeks = []

for root, dirs, files in os.walk('train/'):
    week_files = {}
    
    # Group files by week number
    for file in files:
        if file.endswith('.csv'):
            week_num = file.split('_')[-1].replace('.csv', '')
            
            if week_num not in week_files:
                week_files[week_num] = {}
            
            file_path = os.path.join(root, file)
            
            # Categorize as input or output file
            if 'output' in file:
                week_files[week_num]['output'] = file_path
            else:
                week_files[week_num]['input'] = file_path
    
    # Process each week
    for week_num, paths in week_files.items():
        if 'input' in paths and 'output' in paths:
            # Read input and output files
            df = pd.read_csv(paths['input'])
            df_throw = pd.read_csv(paths['output'])
            
            # Process and merge
            df['play_id_n'] = df.groupby(['ball_land_x', 'ball_land_y', 'play_id']).ngroup()
            
            max_frames = df.groupby(['play_id', 'nfl_id'])['frame_id'].max().reset_index()
            max_frames.columns = ['play_id', 'nfl_id', 'max_frame_id']
            
            df_2_extended = df_throw.merge(max_frames, on=['play_id', 'nfl_id'], how='inner')
            df_2_extended['frame_id'] = df_2_extended['frame_id'] + df_2_extended['max_frame_id']
            
            common_cols = ['nfl_id', 'play_id', 'frame_id', 'x', 'y']
            df_2_subset = df_2_extended[common_cols]
            
            df_combined = pd.concat([df, df_2_subset], ignore_index=True)
            df_combined = df_combined.sort_values(['play_id', 'nfl_id', 'frame_id']).reset_index(drop=True)
            
            all_weeks.append(df_combined)

# Merge all weeks together
final_df = pd.concat(all_weeks, ignore_index=True)
